In [123]:
%load_ext autoreload
%autoreload 2

import os
import glob
from mair_tools import notebook_utils
from mair_tools import papers_processing

from typing import List
from tqdm import tqdm
from spacy.lang.en import English
import pandas as pd

tqdm.pandas()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
INPUT_FOLDER = "../arxiv_dump/raw_extracted_texts/"
OUTPUT_FOLDER = "../arxiv_dump/cleaned_texts/"

MIN_PAPER_LEN = 100
input_files_pattern = os.path.join(INPUT_FOLDER, "*.txt")

# Loading texts

In [145]:
df = notebook_utils.load_texts_as_df(input_files_pattern)

In [157]:
df

,text
2004.02353v2,Adaptive Explainable Neural Networks (AxNNs) \...
2007.14677v1,On the Use of Interpretable Machine Learning f...
2005.06898v2,0\n2\n0\n2\n \ny\na\nM\n8\n1\n \n \n \n]\nL\nC...
2009.06433v1,0\n2\n0\n2\n \np\ne\nS\n4\n1\n \n \n \n]\nC\nH...
2004.04019v1,0\n2\n0\n2\n \nr\np\nA\n8\n \n \n \n]\nT\nO\n....
...,...
1909.12367v1,9\n1\n0\n2\n \np\ne\nS\n6\n2\n \n \n \n]\nG\nL...
2008.04448v1,Explainable Artiﬁcial Intelligence Based Fault...
2001.10056v1,0\n2\n0\n2\n \nn\na\nJ\n \n3\n2\n \n \n]\nG\nL...
2005.10284v4,An Adversarial Approach for Explaining the Pre...


### Removing empty papers

In [158]:
df = df[~(df['text'].str.len()<MIN_PAPER_LEN)]
len(df)

480

# Cleaning

In [159]:
df['cleaned_text']=df.text.progress_apply(papers_processing.clean_text)

100%|██████████| 480/480 [00:00<00:00, 2222.60it/s]


### Finding beginning of paper content

In [160]:
en = English()
df['doc'] = df.cleaned_text.progress_apply(en)

100%|██████████| 480/480 [00:13<00:00, 34.41it/s]


In [184]:
# Not ideal, need to find better solution
BEGINNING_WORDS = ['abstract', 'summary']#, 'introduction']
df['beginning'] = df['doc'].apply(find_first, args=(BEGINNING_WORDS,))
df['clean_stripped_text'] = df.progress_apply(lambda row: row.doc[row.beginning:].text, axis=1)

100%|██████████| 480/480 [00:02<00:00, 215.64it/s]


## Saving

In [185]:
notebook_utils.save_texts_series_to_dir(df['clean_stripped_text'], OUTPUT_FOLDER)